In [2]:
# setup
import sqlite3
print('All good')

All good


### 1. Player

In [3]:
def player_upsert():
    conn = None
    cursor = None

    
    conn = sqlite3.connect('../databases/player.db')
    cursor = conn.cursor()

    cursor.execute('insert into player default values')

    conn.commit()
    conn.close()

### 7. Move

In [39]:
# PARAMETER DEFAULT VALUES
DEFAULT_DESCRIPTION = '' # (okay if it's empty)
DEFAULT_ATTACK_TYPE = ''
DEFAULT_BASE_DAMAGE = -10
DEFAULT_COST = '{}'
DEFAULT_EFFECTS = '{"null": "null"}'

def move_upsert(move_name, description = '', attack_type = '', base_damage = -10, cost = '{}', effects = '{"null": "null"}', force_insert = False):
    conn = None
    cursor = None

    try:
        conn = sqlite3.connect('../databases/move.db')
        cursor = conn.cursor()

        action_type = '' # we need to know whether we're going to be inserting or updating a record into the table
        cursor.execute(f'select count(*) from move where move_name = \"{move_name}\"')

        if cursor.fetchall()[0][0] > 0:
            action_type = 'upsert'
        else:
            action_type = 'insert'

        # action type is insert but not all parameters are populated (except description)
        if action_type == 'insert' and (attack_type == DEFAULT_ATTACK_TYPE or base_damage == DEFAULT_BASE_DAMAGE or cost == DEFAULT_COST or effects == DEFAULT_EFFECTS):
            print('All parameters need to be populated in order to insert into move.db')
            return # exits the function
        
        # action type is insert and all parameters are populated
        if action_type == 'insert' and not (attack_type == DEFAULT_ATTACK_TYPE or base_damage == DEFAULT_BASE_DAMAGE or cost == DEFAULT_COST or effects == DEFAULT_EFFECTS):
            cursor.execute(f'''
                insert into move (move_name, description, attack_type, base_damage, cost, effects)
                values (\'{move_name}\', \'{description}\', \'{attack_type}\', {base_damage}, \'{cost}\', \'{effects}\')
            ''')

        # action type is upsert and force insert is false
        if action_type == 'upsert' and force_insert == False:
            # first getting current values for each attribute
            cursor.execute(f'select * from move where move_name = \"{move_name}\"')
            result = cursor.fetchall()[0]
            curr_description = result[2]
            curr_attack_type = result[3]
            curr_base_damage = result[4]
            curr_cost = result[5]
            curr_effects = result[6]

            # only update the attributes we specified
            if description != DEFAULT_DESCRIPTION:
                curr_description = description
            if attack_type != DEFAULT_ATTACK_TYPE:
                curr_attack_type = attack_type
            if base_damage != DEFAULT_BASE_DAMAGE:
                curr_base_damage = base_damage
            if cost != DEFAULT_COST:
                curr_cost = cost
            if effects != DEFAULT_EFFECTS:
                curr_effects = effects

            # ready to update row
            cursor.execute(f'''
                update move
                set
                    description = \'{curr_description}\',
                    attack_type = \'{curr_attack_type}\',
                    base_damage = {curr_base_damage},
                    cost = \'{curr_cost}\',
                    effects = \'{curr_effects}\'
                where move_name = \'{move_name}\'
            ''')

        # action type is upsert and force insert is true
        if action_type == 'upsert' and force_insert == True:
            print(f'Move {move_name} already in move.db')
            return # exits the function

        conn.commit()
    except Exception as e:
        print(f'SQL ERROR - {e}')
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

### 8. Ability

In [31]:
# PARAMETER DEFAULT VALUES
DEFAULT_DESCRIPTION = '' # (okay if it's empty)
DEFAULT_ACTIVABLE = -1
DEFAULT_ACTIVE_REQ = '{"null": "null"}'
DEFAULT_REPEATABLE = -1
DEFAULT_EFFECTS = '{"null": "null"}'

def ability_upsert(ability_name, description = DEFAULT_DESCRIPTION, activable = DEFAULT_ACTIVABLE, active_req = DEFAULT_ACTIVE_REQ, repeatable = DEFAULT_REPEATABLE, effects = DEFAULT_EFFECTS, force_insert = False):
    conn = None
    cursor = None

    try:
        conn = sqlite3.connect('../databases/ability.db')
        cursor = conn.cursor()

        action_type = '' # we need to know whether we're going to be inserting or updating a record into the table
        cursor.execute(f'select count(*) from ability where ability_name = \"{ability_name}\"')

        if cursor.fetchall()[0][0] > 0:
            action_type = 'upsert'
        else:
            action_type = 'insert'

        # action type is insert but not all parameters are populated (except description)
        if action_type == 'insert' and (activable == DEFAULT_ACTIVABLE or active_req == DEFAULT_ACTIVE_REQ or repeatable == DEFAULT_REPEATABLE or effects == DEFAULT_EFFECTS):
            print ('All parameters need to be populated in order to insert into ability.db')
            return # exits the function

        # action type is insert and all parameters are populated
        if action_type == 'insert' and not (activable == DEFAULT_ACTIVABLE or active_req == DEFAULT_ACTIVE_REQ or repeatable == DEFAULT_REPEATABLE or effects == DEFAULT_EFFECTS):
            print('F1')
            cursor.execute(f'''
                insert into ability(ability_name, description, activable, active_req, repeatable, effects)
                values (\'{ability_name}\', \'{description}\', {activable}, \'{active_req}\', {repeatable}, \'{effects}\')
            ''')

        # action type is upsert and force insert is false
        if action_type == 'upsert' and force_insert == False:
            # first getting current values for each attribute
            cursor.execute(f'select * from ability where ability_name = \"{ability_name}\"')
            result = cursor.fetchall()[0]
            curr_description = result[2]
            curr_activable = result[3]
            curr_active_req = result[4]
            curr_repeatable = result[5]
            curr_effects = result[6]

            # only update the attributes we specified
            if description != DEFAULT_DESCRIPTION:
                curr_description = description
            if activable != DEFAULT_ACTIVABLE:
                curr_activable = activable
            if active_req != DEFAULT_ACTIVE_REQ:
                curr_active_req = active_req
            if repeatable != DEFAULT_REPEATABLE:
                curr_repeatable = repeatable
            if effects != DEFAULT_EFFECTS:
                curr_effects = effects

            # ready to update row
            cursor.execute(f'''
                update ability
                set
                    description = \'{curr_description}\',
                    activable = {curr_activable},
                    active_req = \'{curr_active_req}\',
                    repeatable = {curr_repeatable},
                    effects = \'{curr_effects}\'
            ''')

        # action type is upsert and force insert is true
        if action_type == 'upsert' and force_insert == True:
            print(f'Ability {ability_name} already in ability.db')
            return # exits the function

        conn.commit()
    except Exception as e:
        print(f'SQL ERROR - {e}')
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [41]:
# test ground here

name = ''
description = ''
type = ''
base_damage = 0
cost = '{}'
effects = '{}'

name = 'Bug Bite'
description = ''
type = 'grass'
base_damage = 30
cost = '{"grass": 1}'
effects = '{}'

move_upsert(name, description, type, base_damage, cost, effects, True)
# COMING BACK: Fix some moves can have variations (ex. Bug Bite can have different costs)

Move Bug Bite already in move.db


In [33]:
# test ground here

name = ''
description = ''
activable = 0
active_req = '{}'
repeatable = 0
effects = '{}'

name = 'Powder Heal'
description = 'Once during your turn, you may heal 20 damage from each of your Pokemon'
activable = 1
active_req = '{}'
repeatable = 0
effects = '{"heal_all": 20}'

ability_upsert(name, description, activable, active_req, repeatable, effects, True)

Ability Powder Heal already in ability.db
